# Day 6: Market Making

## Week 17 - Options & Deep Hedging

**Date:** January 23, 2026

---

## Learning Objectives

1. **Understand market maker dynamics** - Role, risks, and profit mechanisms
2. **Inventory management** - Optimal inventory control and risk mitigation
3. **Bid-ask spread optimization** - Avellaneda-Stoikov model and extensions
4. **Simulation** - Build a complete market making simulator

---

## Key Concepts

### What is a Market Maker?
- Provides liquidity by continuously quoting bid/ask prices
- Profits from the bid-ask spread
- Bears inventory risk from adverse selection

### Key Risks
- **Inventory Risk**: Holding unwanted positions
- **Adverse Selection**: Trading against informed traders
- **Execution Risk**: Order flow uncertainty

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

---

## Part 1: Market Dynamics Simulation

### Mid-Price Process

We model the mid-price $S_t$ as a geometric Brownian motion:

$$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Discretized:
$$S_{t+\Delta t} = S_t \exp\left((\mu - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t} Z\right)$$

where $Z \sim \mathcal{N}(0,1)$

In [ ]:
@dataclass
class MarketParams:
    """Market simulation parameters"""
    S0: float = 100.0          # Initial price
    mu: float = 0.0            # Drift (assume zero for MM)
    sigma: float = 0.02        # Volatility (per time unit)
    dt: float = 1/252/390      # Time step (1 minute in trading day)
    lambda_buy: float = 1.0    # Base buy arrival rate
    lambda_sell: float = 1.0   # Base sell arrival rate
    k: float = 1.5             # Order arrival sensitivity to spread


def simulate_mid_price(params: MarketParams, n_steps: int) -> np.ndarray:
    """Simulate mid-price using GBM"""
    prices = np.zeros(n_steps)
    prices[0] = params.S0
    
    # Pre-generate random shocks
    Z = np.random.standard_normal(n_steps - 1)
    
    # Simulate path
    drift = (params.mu - 0.5 * params.sigma**2) * params.dt
    diffusion = params.sigma * np.sqrt(params.dt)
    
    for t in range(1, n_steps):
        prices[t] = prices[t-1] * np.exp(drift + diffusion * Z[t-1])
    
    return prices


# Simulate one day of trading (390 minutes)
params = MarketParams()
n_steps = 390
mid_prices = simulate_mid_price(params, n_steps)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(mid_prices, 'b-', linewidth=1)
ax.set_xlabel('Time (minutes)')
ax.set_ylabel('Mid Price ($)')
ax.set_title('Simulated Mid-Price Path (1 Trading Day)')
ax.axhline(y=params.S0, color='r', linestyle='--', alpha=0.5, label=f'Initial: ${params.S0}')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Price range: ${mid_prices.min():.2f} - ${mid_prices.max():.2f}")
print(f"Final return: {(mid_prices[-1]/mid_prices[0] - 1)*100:.3f}%")

---

## Part 2: Avellaneda-Stoikov Market Making Model

### Key Formulas

The **reservation price** (fair value adjusted for inventory):

$$r(s, q, t) = s - q \gamma \sigma^2 (T - t)$$

The **optimal spread**:

$$\delta^* = \gamma \sigma^2 (T-t) + \frac{2}{\gamma} \ln\left(1 + \frac{\gamma}{k}\right)$$

Where:
- $s$ = current mid-price
- $q$ = current inventory
- $\gamma$ = risk aversion parameter
- $\sigma$ = volatility
- $T$ = terminal time
- $k$ = order arrival intensity parameter

In [ ]:
@dataclass
class AvellanedaStoikovParams:
    """Avellaneda-Stoikov model parameters"""
    gamma: float = 0.1         # Risk aversion
    sigma: float = 0.02        # Volatility
    k: float = 1.5             # Order arrival intensity
    T: float = 1.0             # Terminal time (normalized)
    q_max: int = 10            # Max inventory limit


class AvellanedaStoikovMM:
    """Avellaneda-Stoikov Market Making Model"""
    
    def __init__(self, params: AvellanedaStoikovParams):
        self.params = params
    
    def reservation_price(self, mid_price: float, inventory: int, 
                          time_remaining: float) -> float:
        """Calculate reservation price adjusted for inventory"""
        p = self.params
        return mid_price - inventory * p.gamma * p.sigma**2 * time_remaining
    
    def optimal_spread(self, time_remaining: float) -> float:
        """Calculate optimal total spread"""
        p = self.params
        term1 = p.gamma * p.sigma**2 * time_remaining
        term2 = (2/p.gamma) * np.log(1 + p.gamma/p.k)
        return term1 + term2
    
    def get_quotes(self, mid_price: float, inventory: int, 
                   time_remaining: float) -> Tuple[float, float]:
        """Calculate bid and ask prices"""
        r = self.reservation_price(mid_price, inventory, time_remaining)
        delta = self.optimal_spread(time_remaining)
        
        bid = r - delta/2
        ask = r + delta/2
        
        # Ensure bid < mid < ask (market integrity)
        bid = min(bid, mid_price - 0.01)
        ask = max(ask, mid_price + 0.01)
        
        return bid, ask


# Demonstrate the model
as_params = AvellanedaStoikovParams()
mm = AvellanedaStoikovMM(as_params)

# Test with different inventory levels
mid_price = 100.0
time_remaining = 0.5

print("Avellaneda-Stoikov Quote Adjustment")
print("=" * 50)
print(f"Mid-price: ${mid_price:.2f}")
print(f"Time remaining: {time_remaining:.2f}")
print()

inventories = [-5, -2, 0, 2, 5]
for q in inventories:
    bid, ask = mm.get_quotes(mid_price, q, time_remaining)
    r = mm.reservation_price(mid_price, q, time_remaining)
    spread = ask - bid
    print(f"Inventory={q:+3d}: Bid=${bid:.4f}, Ask=${ask:.4f}, "
          f"Spread=${spread:.4f}, Reservation=${r:.4f}")

In [ ]:
# Visualize quote skewing based on inventory
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Bid-Ask quotes vs inventory
inventories = np.arange(-10, 11)
bids = []
asks = []
reservations = []

for q in inventories:
    bid, ask = mm.get_quotes(mid_price, q, time_remaining)
    r = mm.reservation_price(mid_price, q, time_remaining)
    bids.append(bid)
    asks.append(ask)
    reservations.append(r)

ax1 = axes[0]
ax1.fill_between(inventories, bids, asks, alpha=0.3, color='blue', label='Spread')
ax1.plot(inventories, bids, 'g-', linewidth=2, label='Bid')
ax1.plot(inventories, asks, 'r-', linewidth=2, label='Ask')
ax1.plot(inventories, reservations, 'k--', linewidth=2, label='Reservation')
ax1.axhline(y=mid_price, color='gray', linestyle=':', alpha=0.7, label='Mid-price')
ax1.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
ax1.set_xlabel('Inventory')
ax1.set_ylabel('Price ($)')
ax1.set_title('Quote Skewing Based on Inventory')
ax1.legend(loc='upper right')
ax1.set_xlim(-10, 10)

# Plot 2: Optimal spread vs time remaining
times = np.linspace(0.01, 1.0, 100)
spreads = [mm.optimal_spread(t) for t in times]

ax2 = axes[1]
ax2.plot(times, spreads, 'b-', linewidth=2)
ax2.set_xlabel('Time Remaining (normalized)')
ax2.set_ylabel('Optimal Spread ($)')
ax2.set_title('Optimal Spread vs Time to End of Day')
ax2.fill_between(times, 0, spreads, alpha=0.2)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("• Long inventory (q>0) → Lower bid/ask (want to sell)")
print("• Short inventory (q<0) → Higher bid/ask (want to buy)")
print("• Spread decreases as time remaining decreases")

---

## Part 3: Order Arrival Model

### Poisson Process for Order Arrivals

The arrival rate of buy/sell orders depends on how competitive the quotes are:

$$\lambda^{bid}(\delta^b) = A \exp(-k \delta^b)$$
$$\lambda^{ask}(\delta^a) = A \exp(-k \delta^a)$$

Where:
- $\delta^b = S - bid$ (distance from mid to bid)
- $\delta^a = ask - S$ (distance from mid to ask)
- $A$ = baseline arrival intensity
- $k$ = arrival sensitivity

In [ ]:
class OrderArrivalModel:
    """Poisson-based order arrival model"""
    
    def __init__(self, A: float = 10.0, k: float = 1.5):
        self.A = A  # Base arrival rate
        self.k = k  # Sensitivity to spread
    
    def arrival_rate(self, delta: float) -> float:
        """Calculate arrival rate given distance from mid"""
        return self.A * np.exp(-self.k * delta)
    
    def probability_of_fill(self, delta: float, dt: float) -> float:
        """Probability of fill in time interval dt"""
        rate = self.arrival_rate(delta)
        return 1 - np.exp(-rate * dt)
    
    def simulate_fills(self, bid: float, ask: float, 
                       mid_price: float, dt: float) -> Tuple[bool, bool]:
        """Simulate whether bid/ask get filled"""
        delta_bid = mid_price - bid
        delta_ask = ask - mid_price
        
        p_bid_fill = self.probability_of_fill(delta_bid, dt)
        p_ask_fill = self.probability_of_fill(delta_ask, dt)
        
        bid_filled = np.random.random() < p_bid_fill
        ask_filled = np.random.random() < p_ask_fill
        
        return bid_filled, ask_filled


# Visualize fill probability
arrival_model = OrderArrivalModel(A=10.0, k=1.5)

deltas = np.linspace(0.01, 1.0, 100)
rates = [arrival_model.arrival_rate(d) for d in deltas]
probs = [arrival_model.probability_of_fill(d, 0.1) for d in deltas]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(deltas, rates, 'b-', linewidth=2)
ax1.set_xlabel('Distance from Mid-Price ($)')
ax1.set_ylabel('Arrival Rate (λ)')
ax1.set_title('Order Arrival Rate vs Quote Distance')
ax1.fill_between(deltas, 0, rates, alpha=0.2)

ax2 = axes[1]
ax2.plot(deltas, probs, 'r-', linewidth=2)
ax2.set_xlabel('Distance from Mid-Price ($)')
ax2.set_ylabel('Fill Probability')
ax2.set_title('Fill Probability vs Quote Distance (dt=0.1)')
ax2.fill_between(deltas, 0, probs, alpha=0.2, color='red')

plt.tight_layout()
plt.show()

print("\n📊 Trade-off:")
print("• Tight spreads → Higher fill rate, lower profit per trade")
print("• Wide spreads → Lower fill rate, higher profit per trade")

---

## Part 4: Complete Market Making Simulator

Now we combine everything into a full simulation.

In [ ]:
@dataclass
class MMState:
    """Market maker state at each time step"""
    time: int
    mid_price: float
    inventory: int
    cash: float
    bid: float
    ask: float
    pnl: float


class MarketMakingSimulator:
    """Complete market making simulation"""
    
    def __init__(self, 
                 market_params: MarketParams,
                 as_params: AvellanedaStoikovParams,
                 arrival_A: float = 10.0,
                 arrival_k: float = 1.5):
        
        self.market_params = market_params
        self.mm = AvellanedaStoikovMM(as_params)
        self.arrival_model = OrderArrivalModel(arrival_A, arrival_k)
        self.q_max = as_params.q_max
        
    def run(self, n_steps: int = 390) -> List[MMState]:
        """Run the market making simulation"""
        # Generate price path
        mid_prices = simulate_mid_price(self.market_params, n_steps)
        
        # Initialize state
        inventory = 0
        cash = 0.0
        history = []
        
        for t in range(n_steps):
            mid_price = mid_prices[t]
            time_remaining = (n_steps - t) / n_steps
            
            # Get optimal quotes
            bid, ask = self.mm.get_quotes(mid_price, inventory, time_remaining)
            
            # Skip quoting if at inventory limits
            quote_bid = inventory < self.q_max
            quote_ask = inventory > -self.q_max
            
            if not quote_bid:
                bid = 0  # No bid
            if not quote_ask:
                ask = float('inf')  # No ask
            
            # Simulate order arrivals
            if t < n_steps - 1:  # Don't trade on last step
                bid_filled, ask_filled = self.arrival_model.simulate_fills(
                    bid, ask, mid_price, self.market_params.dt
                )
                
                # Update inventory and cash
                if bid_filled and quote_bid:
                    inventory += 1
                    cash -= bid
                
                if ask_filled and quote_ask:
                    inventory -= 1
                    cash += ask
            
            # Calculate mark-to-market PnL
            pnl = cash + inventory * mid_price
            
            # Record state
            state = MMState(
                time=t,
                mid_price=mid_price,
                inventory=inventory,
                cash=cash,
                bid=bid if quote_bid else np.nan,
                ask=ask if quote_ask else np.nan,
                pnl=pnl
            )
            history.append(state)
        
        return history


# Run simulation
market_params = MarketParams(S0=100.0, sigma=0.02)
as_params = AvellanedaStoikovParams(gamma=0.1, sigma=0.02, k=1.5, q_max=10)

simulator = MarketMakingSimulator(market_params, as_params)
history = simulator.run(n_steps=390)

# Extract results
times = [s.time for s in history]
prices = [s.mid_price for s in history]
inventories = [s.inventory for s in history]
pnls = [s.pnl for s in history]
bids = [s.bid for s in history]
asks = [s.ask for s in history]

print(f"Simulation completed!")
print(f"Final inventory: {history[-1].inventory}")
print(f"Final PnL: ${history[-1].pnl:.2f}")

In [ ]:
# Comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Price with bid-ask quotes
ax1 = axes[0, 0]
ax1.plot(times, prices, 'b-', linewidth=1, label='Mid Price', alpha=0.8)
ax1.fill_between(times, bids, asks, alpha=0.2, color='green', label='Bid-Ask Spread')
ax1.set_xlabel('Time (minutes)')
ax1.set_ylabel('Price ($)')
ax1.set_title('Mid-Price and Quoted Spread')
ax1.legend()

# Plot 2: Inventory over time
ax2 = axes[0, 1]
ax2.fill_between(times, 0, inventories, alpha=0.5, 
                  where=[i >= 0 for i in inventories], color='green', label='Long')
ax2.fill_between(times, 0, inventories, alpha=0.5,
                  where=[i < 0 for i in inventories], color='red', label='Short')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.axhline(y=as_params.q_max, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=-as_params.q_max, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Time (minutes)')
ax2.set_ylabel('Inventory')
ax2.set_title('Inventory Over Time')
ax2.legend()

# Plot 3: PnL over time
ax3 = axes[1, 0]
ax3.plot(times, pnls, 'purple', linewidth=1.5)
ax3.fill_between(times, 0, pnls, alpha=0.3, 
                  where=[p >= 0 for p in pnls], color='green')
ax3.fill_between(times, 0, pnls, alpha=0.3,
                  where=[p < 0 for p in pnls], color='red')
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_xlabel('Time (minutes)')
ax3.set_ylabel('PnL ($)')
ax3.set_title('Mark-to-Market PnL')

# Plot 4: Spread over time
spreads = [a - b for a, b in zip(asks, bids) if not np.isnan(a) and not np.isnan(b)]
valid_times = [t for t, a, b in zip(times, asks, bids) if not np.isnan(a) and not np.isnan(b)]

ax4 = axes[1, 1]
ax4.plot(valid_times, spreads, 'orange', linewidth=1)
ax4.fill_between(valid_times, 0, spreads, alpha=0.3, color='orange')
ax4.set_xlabel('Time (minutes)')
ax4.set_ylabel('Spread ($)')
ax4.set_title('Bid-Ask Spread Over Time')

plt.tight_layout()
plt.show()

---

## Part 5: Monte Carlo Analysis

Run multiple simulations to analyze performance distribution.

In [ ]:
def run_monte_carlo(n_simulations: int = 500, n_steps: int = 390) -> dict:
    """Run multiple simulations and collect statistics"""
    
    final_pnls = []
    final_inventories = []
    max_drawdowns = []
    sharpe_ratios = []
    
    market_params = MarketParams(S0=100.0, sigma=0.02)
    as_params = AvellanedaStoikovParams(gamma=0.1, sigma=0.02, k=1.5, q_max=10)
    
    for i in range(n_simulations):
        simulator = MarketMakingSimulator(market_params, as_params)
        history = simulator.run(n_steps=n_steps)
        
        pnls = [s.pnl for s in history]
        final_pnls.append(history[-1].pnl)
        final_inventories.append(history[-1].inventory)
        
        # Max drawdown
        cummax = np.maximum.accumulate(pnls)
        drawdowns = (cummax - pnls) / (np.abs(cummax) + 1e-8)
        max_drawdowns.append(np.max(drawdowns))
        
        # Simple Sharpe approximation
        returns = np.diff(pnls)
        if np.std(returns) > 0:
            sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252 * 390)
        else:
            sharpe = 0
        sharpe_ratios.append(sharpe)
    
    return {
        'pnls': np.array(final_pnls),
        'inventories': np.array(final_inventories),
        'drawdowns': np.array(max_drawdowns),
        'sharpes': np.array(sharpe_ratios)
    }


print("Running Monte Carlo simulation (500 trials)...")
results = run_monte_carlo(n_simulations=500)
print("Done!\n")

# Statistics
print("=" * 50)
print("MONTE CARLO RESULTS (500 Simulations)")
print("=" * 50)
print(f"\nPnL Statistics:")
print(f"  Mean:   ${results['pnls'].mean():.2f}")
print(f"  Std:    ${results['pnls'].std():.2f}")
print(f"  Median: ${np.median(results['pnls']):.2f}")
print(f"  Min:    ${results['pnls'].min():.2f}")
print(f"  Max:    ${results['pnls'].max():.2f}")
print(f"  5th %:  ${np.percentile(results['pnls'], 5):.2f}")
print(f"  95th %: ${np.percentile(results['pnls'], 95):.2f}")

print(f"\nWin Rate: {(results['pnls'] > 0).mean()*100:.1f}%")
print(f"Mean Sharpe Ratio: {results['sharpes'].mean():.2f}")

In [ ]:
# Visualize Monte Carlo results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# PnL Distribution
ax1 = axes[0, 0]
ax1.hist(results['pnls'], bins=50, edgecolor='black', alpha=0.7, color='blue')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
ax1.axvline(x=results['pnls'].mean(), color='green', linestyle='-', 
            linewidth=2, label=f"Mean: ${results['pnls'].mean():.2f}")
ax1.set_xlabel('Final PnL ($)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Final PnL')
ax1.legend()

# Final Inventory Distribution
ax2 = axes[0, 1]
ax2.hist(results['inventories'], bins=21, edgecolor='black', alpha=0.7, color='orange')
ax2.axvline(x=0, color='green', linestyle='--', linewidth=2, label='Neutral')
ax2.set_xlabel('Final Inventory')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Final Inventory')
ax2.legend()

# Sharpe Ratio Distribution
ax3 = axes[1, 0]
ax3.hist(results['sharpes'], bins=50, edgecolor='black', alpha=0.7, color='purple')
ax3.axvline(x=results['sharpes'].mean(), color='red', linestyle='-',
            linewidth=2, label=f"Mean: {results['sharpes'].mean():.2f}")
ax3.set_xlabel('Sharpe Ratio')
ax3.set_ylabel('Frequency')
ax3.set_title('Distribution of Sharpe Ratios')
ax3.legend()

# PnL vs Final Inventory scatter
ax4 = axes[1, 1]
scatter = ax4.scatter(results['inventories'], results['pnls'], 
                       alpha=0.5, c=results['sharpes'], cmap='RdYlGn')
ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax4.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax4.set_xlabel('Final Inventory')
ax4.set_ylabel('Final PnL ($)')
ax4.set_title('PnL vs Final Inventory')
plt.colorbar(scatter, ax=ax4, label='Sharpe')

plt.tight_layout()
plt.show()

---

## Part 6: Risk Aversion Sensitivity Analysis

How does the risk aversion parameter $\gamma$ affect performance?

In [ ]:
def sensitivity_analysis(gammas: list, n_sims: int = 100) -> dict:
    """Analyze sensitivity to risk aversion parameter"""
    
    results = {g: {'pnls': [], 'inventories': [], 'trades': []} for g in gammas}
    
    for gamma in gammas:
        print(f"Testing gamma = {gamma}...")
        
        market_params = MarketParams(S0=100.0, sigma=0.02)
        as_params = AvellanedaStoikovParams(gamma=gamma, sigma=0.02, k=1.5, q_max=10)
        
        for _ in range(n_sims):
            simulator = MarketMakingSimulator(market_params, as_params)
            history = simulator.run(n_steps=390)
            
            results[gamma]['pnls'].append(history[-1].pnl)
            results[gamma]['inventories'].append(history[-1].inventory)
            
            # Count trades (inventory changes)
            invs = [s.inventory for s in history]
            trades = sum(1 for i in range(1, len(invs)) if invs[i] != invs[i-1])
            results[gamma]['trades'].append(trades)
    
    return results


gammas = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
sensitivity_results = sensitivity_analysis(gammas, n_sims=100)
print("Done!")

In [ ]:
# Visualize sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

mean_pnls = [np.mean(sensitivity_results[g]['pnls']) for g in gammas]
std_pnls = [np.std(sensitivity_results[g]['pnls']) for g in gammas]
mean_trades = [np.mean(sensitivity_results[g]['trades']) for g in gammas]
mean_abs_inv = [np.mean(np.abs(sensitivity_results[g]['inventories'])) for g in gammas]

# Mean PnL vs gamma
ax1 = axes[0]
ax1.errorbar(gammas, mean_pnls, yerr=std_pnls, marker='o', capsize=5, linewidth=2)
ax1.set_xlabel('Risk Aversion (γ)')
ax1.set_ylabel('Mean PnL ($)')
ax1.set_title('Mean PnL vs Risk Aversion')
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)

# Mean trades vs gamma
ax2 = axes[1]
ax2.plot(gammas, mean_trades, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Risk Aversion (γ)')
ax2.set_ylabel('Mean Number of Trades')
ax2.set_title('Trade Frequency vs Risk Aversion')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)

# Mean abs inventory vs gamma
ax3 = axes[2]
ax3.plot(gammas, mean_abs_inv, 'ro-', linewidth=2, markersize=8)
ax3.set_xlabel('Risk Aversion (γ)')
ax3.set_ylabel('Mean |Final Inventory|')
ax3.set_title('Inventory Risk vs Risk Aversion')
ax3.set_xscale('log')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 70)
print("SENSITIVITY ANALYSIS SUMMARY")
print("=" * 70)
print(f"{'Gamma':>8} | {'Mean PnL':>12} | {'Std PnL':>12} | {'Trades':>10} | {'|Inv|':>8}")
print("-" * 70)
for g in gammas:
    mp = np.mean(sensitivity_results[g]['pnls'])
    sp = np.std(sensitivity_results[g]['pnls'])
    mt = np.mean(sensitivity_results[g]['trades'])
    mi = np.mean(np.abs(sensitivity_results[g]['inventories']))
    print(f"{g:>8.2f} | ${mp:>10.2f} | ${sp:>10.2f} | {mt:>10.1f} | {mi:>8.2f}")

---

## Part 7: Interview Questions & Key Takeaways

### Common Interview Questions

1. **What is the role of a market maker?**
   - Provide liquidity by continuously quoting bid/ask prices
   - Profit from bid-ask spread
   - Manage inventory risk and adverse selection

2. **Explain the Avellaneda-Stoikov model**
   - Reservation price adjusts for inventory: $r = s - q\gamma\sigma^2(T-t)$
   - Optimal spread balances inventory risk and execution probability
   - Higher risk aversion → wider spreads, less trading

3. **What is adverse selection?**
   - Trading against informed traders who know future price direction
   - Market maker systematically loses on trades with informed traders
   - Mitigated by widening spreads, reducing quote size

4. **How does inventory affect quotes?**
   - Long inventory → skew quotes down (lower bid/ask to sell)
   - Short inventory → skew quotes up (higher bid/ask to buy)
   - Goal: mean-revert inventory to zero

5. **What are the key risks in market making?**
   - Inventory risk (price moves against position)
   - Adverse selection (informed traders)
   - Execution risk (uncertain fills)
   - Technology risk (latency, system failures)

In [ ]:
# Quick reference implementations for interviews

def simple_mm_quote(mid_price: float, inventory: int, 
                    gamma: float = 0.1, sigma: float = 0.02,
                    T: float = 1.0) -> Tuple[float, float]:
    """
    Simple Avellaneda-Stoikov quoting function.
    
    Args:
        mid_price: Current mid-price
        inventory: Current inventory position
        gamma: Risk aversion parameter
        sigma: Price volatility
        T: Time remaining (normalized)
    
    Returns:
        (bid, ask) prices
    """
    # Reservation price (inventory-adjusted fair value)
    reservation = mid_price - inventory * gamma * sigma**2 * T
    
    # Optimal spread
    k = 1.5  # arrival intensity parameter
    spread = gamma * sigma**2 * T + (2/gamma) * np.log(1 + gamma/k)
    
    bid = reservation - spread/2
    ask = reservation + spread/2
    
    return bid, ask


# Example usage
print("Interview-Ready Function Demo:")
print("=" * 40)
bid, ask = simple_mm_quote(100.0, inventory=5)
print(f"Mid: $100.00, Inventory: +5")
print(f"Bid: ${bid:.4f}, Ask: ${ask:.4f}")
print(f"Spread: ${ask-bid:.4f}")
print(f"\n→ Quotes skewed DOWN to reduce long position")

---

## Summary

### Key Formulas

| Concept | Formula |
|---------|--------|
| Reservation Price | $r = s - q\gamma\sigma^2(T-t)$ |
| Optimal Spread | $\delta^* = \gamma\sigma^2(T-t) + \frac{2}{\gamma}\ln(1 + \frac{\gamma}{k})$ |
| Arrival Rate | $\lambda(\delta) = A e^{-k\delta}$ |
| Fill Probability | $P(fill) = 1 - e^{-\lambda \Delta t}$ |

### Key Insights

1. **Quote Skewing**: Adjust quotes based on inventory to mean-revert
2. **Risk-Return Trade-off**: Tighter spreads = more trades but more risk
3. **Time Decay**: Spreads narrow as time remaining decreases
4. **Parameter Tuning**: γ controls the risk-return trade-off

### Next Steps

- Add adverse selection modeling
- Implement multi-asset market making
- Study Almgren-Chriss optimal execution
- Explore deep reinforcement learning for market making